In [1]:
import mbuild as mb

import hoomd
import rowan
import numpy as np

In [2]:
# chains = [mb.Compound(name="A", mass=1) for i in range(2)]
# system = mb.packing.fill_box(
#             compound=chains,
#             n_compounds=[1 for i in range(len(chains))],
#             density=0.01,
#             overlap=0.5,
#             edge=1.5,
#             fix_orientation=True
#         )

# init_snap = hoomd.Snapshot()
# # Create place holder spots in the snapshot for rigid centers
# init_snap.particles.types =["A"]
# init_snap.particles.N = 2
# init_snap.particles.position[:] = [p.pos for p in system.particles()]

# init_snap.configuration.box = [system.box.Lx, system.box.Ly, system.box.Lz, 0, 0,  0 ]

In [3]:
import gsd.hoomd
rigid_log_path = '/home/marjanalbooyeh/logs/pps_rigid/2023-03-03-14:04:17/sim_traj.gsd'
rigid_gsd = gsd.hoomd.open(rigid_log_path)

init_snap = hoomd.Snapshot()
# Create place holder spots in the snapshot for rigid centers
init_snap.particles.types =["A"]
init_snap.particles.N = 2
init_snap.particles.position[:] = rigid_gsd[0].particles.position[0:2]

init_snap.configuration.box = rigid_gsd[0].configuration.box



In [4]:
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device=device)

In [28]:
print(device)

In [5]:
## Add moment of inertia

mass = 1
I = np.zeros(shape=(3, 3))
for r in init_snap.particles.position:
    I += mass * (np.dot(r, r) * np.identity(3) - np.outer(r, r))
I

array([[ 4.89505639e-02, -4.34400668e-05,  6.67155062e-03],
       [-4.34400668e-05,  1.97911186e+00,  3.18715354e-04],
       [ 6.67155062e-03,  3.18715354e-04,  1.93016545e+00]])

In [6]:
init_snap.particles.moment_inertia[:] = [I[0, 0], I[1, 1], I[2, 2]]

In [7]:
sim.create_state_from_snapshot(init_snap)

In [8]:
import torch.nn as nn
import hoomd
import torch
import numpy as np

class NN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, n_layers, act_fn="ReLU"):
        super(NN, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.n_layers = n_layers
        self.act_fn = act_fn

        self.force_net = nn.Sequential(*self._get_net())
        self.torque_net = nn.Sequential(*self._get_net())

    def _get_act_fn(self):
        act = getattr(nn, self.act_fn)
        return act()

    def _get_net(self):
        layers = [nn.Linear(self.in_dim, self.hidden_dim), self._get_act_fn()]
        for i in range(self.n_layers - 1):
            layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
            layers.append(nn.Dropout(p=0.5))
            layers.append(self._get_act_fn())
        layers.append(nn.Linear(self.hidden_dim, self.out_dim))
        return layers

    def forward(self, x):
        return self.force_net(x), self.torque_net(x)


class EllipsCustomForce(hoomd.md.force.Custom):
    def __init__(self, rigid_ids, model_path, in_dim, hidden_dim, out_dim, n_layers, act_fn):
        super().__init__(aniso=True)
        # load ML model
        self.rigid_ids = rigid_ids
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = NN(in_dim=in_dim, hidden_dim=hidden_dim, out_dim=out_dim, n_layers=n_layers, act_fn=act_fn)
        self.model.to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()

    def set_forces(self, timestep):
        # get positions and orientations
        with self._state.cpu_local_snapshot as snap:
            rigid_rtags = snap.particles.rtag[self.rigid_ids]
            positions = np.array(snap.particles.position[rigid_rtags], copy=True)
            orientations = np.array(snap.particles.orientation[rigid_rtags], copy=True)

        rel_positions = np.zeros((positions.shape[0], positions.shape[1] +1))
        rel_positions[0, :3] = positions[0, :3] - positions[ 1, :3]
        rel_positions[1, :3] = positions[1, :3] - positions[ 0, :3]
        rel_positions[:, 3] = np.linalg.norm(rel_positions[:, :3], axis=1)

        rel_orientations = np.zeros((orientations.shape[0], orientations.shape[1]+1))
        rel_orientations[0, :4] = rowan.geometry.riemann_log_map(orientations[0,:], orientations[1, :])
        rel_orientations[1, :4] = rowan.geometry.riemann_log_map(orientations[1, :], orientations[0, :])
        rel_orientations[0, 4] = rowan.geometry.sym_intrinsic_distance(orientations[ 0, :], orientations [1, :])
        rel_orientations[1, 4] = rowan.geometry.sym_intrinsic_distance(orientations[1, :], orientations[0, :])
        
        positions_tensor = torch.from_numpy(rel_positions).type(torch.FloatTensor)
        orientations_tensor = torch.from_numpy(rel_orientations).type(torch.FloatTensor)
        model_input = torch.cat((positions_tensor, orientations_tensor), 1).to(self.device)
#         print('****************************************')
#         print('orients: ', orientations)
#         print('pos: ', positions)
#         print('rel orients: ', orientations_tensor)
#         print('rel pos: ', rel_positions)
#         print('model inp: ', model_input)

        force_prediction, torque_prediction = self.model(model_input)
        predicted_force = force_prediction.cpu().detach().numpy()
        predicted_torque = torque_prediction.cpu().detach().numpy()
#         print('predicted force: ', predicted_force)
#         print('predicted torque: ', predicted_torque)
        self.predicted_force = predicted_force
        self.predicted_torque = predicted_torque
        with self.cpu_local_force_arrays as arrays:

#             print('timestep: ', timestep)
#             print('****************************************')
#             print(arrays.potential_energy)
            arrays.force[rigid_rtags] = predicted_force
            arrays.torque[rigid_rtags] = predicted_torque

In [9]:
forcefields = []
ml_model_dir= '/home/marjanalbooyeh/logs/ML/2023-03-01 16:57:39/model/best_model.pth'



in_dim=7
out_dim=3
hidden_dim=64
n_layers=2
act_fn="Tanh"
custom_force = EllipsCustomForce(rigid_ids=[0, 1], 
                                 model_path=ml_model_dir, in_dim=9,
                                 hidden_dim=hidden_dim, out_dim=3,
                                 n_layers=n_layers, act_fn=act_fn)
forcefields.append(custom_force)

In [10]:
_all = hoomd.filter.All()

In [11]:
## set parameters

dt = 0.0003
tau_kT = 10
kT = 1.5

In [12]:
integrator_method = hoomd.md.methods.NVT(filter=_all, kT=kT, tau=tau_kT)
integrator = hoomd.md.Integrator(dt=dt, integrate_rotational_dof=True)
integrator.forces = forcefields
sim.operations.add(integrator)

In [13]:
sim.operations.integrator.methods = [integrator_method]

In [14]:
sim.state.thermalize_particle_momenta(filter=_all, kT=kT)

*Warning*: Simulation.seed is not set, using default seed=0


In [15]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=_all)
sim.operations.computes.append(thermodynamic_properties)

In [16]:
import os
from datetime import datetime
log_dir = "/home/marjanalbooyeh/logs/pps_CG/"
now = datetime.now()
log_path = os.path.join(log_dir, now.strftime("%Y-%m-%d-%H:%M:%S")) + '/'


if not os.path.exists(log_path):
    os.mkdir(log_path)
    
log_quantities = [
            "kinetic_temperature",
            "potential_energy",
            "kinetic_energy",
            "volume",
            "pressure",
            "pressure_tensor"
        ]
logger = hoomd.logging.Logger(categories=["scalar", "string", "particle"])
logger.add(sim, quantities=["timestep", "tps"])
thermo_props = thermodynamic_properties
logger.add(thermo_props, quantities=log_quantities)
# logger = hoomd.logging.Logger(categories=["scalar", "string", "particle"])
for f in forcefields:
    logger.add(f, quantities=["forces", "torques"])

# table_file = hoomd.write.Table(
#     output=open(os.path.join(log_path, "sim_traj.txt"), mode="w", newline="\n"),
#     trigger=hoomd.trigger.Periodic(period=int(1000)),
#     logger=logger,
#     max_header_len=None,
# )
gsd_writer = hoomd.write.GSD(
        filename=os.path.join(log_path, "ML_sim_traj.gsd"),
        trigger=hoomd.trigger.Periodic(
            period=int(1000), phase=0
        ),
        mode="wb",
        filter=hoomd.filter.All(),
        dynamic=["momentum"],
        log=logger
)

In [17]:
f.loggables

{'energy': 'scalar',
 'energies': 'particle',
 'additional_energy': 'scalar',
 'forces': 'particle',
 'torques': 'particle',
 'virials': 'particle',
 'additional_virial': 'sequence'}

In [18]:
sim.operations.writers.append(gsd_writer)
# sim.operations.writers.append(table_file)

In [19]:
sim.run(0, write_at_start=True)

In [129]:
with sim._state.cpu_local_snapshot as snap:
    print(snap.particles.net_force)
    print(snap.particles.net_torque)
    print(snap.particles.position)
    print(snap.particles.orientation)

HOOMDArray([[ 1.97188222 -0.20289414  0.06983446]
 [-1.9971689   0.25731409 -0.07120445]])
HOOMDArray([[-1.77881997e-02 -1.32438332e-01  8.45176692e-05]
 [-1.75892152e-02 -2.04224586e-02  6.46268502e-02]])
HOOMDArray([[-1.00347698 -0.00101863  0.15644246]
 [ 0.96083158 -0.00101863  0.15644246]])
HOOMDArray([[1. 0. 0. 0.]
 [1. 0. 0. 0.]])


In [130]:
forcefields[0].forces

array([[ 1.97188222, -0.20289414,  0.06983446],
       [-1.9971689 ,  0.25731409, -0.07120445]])

In [20]:
sim.run(2e6, write_at_start=True)

In [ ]:
operations

In [26]:
import gsd.hoomd
with gsd.hoomd.open(os.path.join(log_path, "ellipsoids_new.gsd"), "wb") as new_t:
        with gsd.hoomd.open(os.path.join(log_path, "ML_sim_traj.gsd")) as old_t:
            for snap in old_t:
                snap.particles.type_shapes = [
                    {
                        "type": "Ellipsoid",
                        "a": 1,
                        "b": 0.5,
                        "c":0.5
                    },

                ]
                snap.validate()
                new_t.append(snap)

In [25]:
log_path

'/home/marjanalbooyeh/logs/pps_CG/2023-03-08-17:05:21/'

In [21]:
import gsd.hoomd
traj = gsd.hoomd.open(os.path.join(log_path, "ML_sim_traj.gsd"))

In [22]:
traj[0].log

{'Simulation/timestep': array([0]),
 'Simulation/tps': array([0.]),
 'md/compute/ThermodynamicQuantities/kinetic_temperature': array([1.0285419]),
 'md/compute/ThermodynamicQuantities/potential_energy': array([0.]),
 'md/compute/ThermodynamicQuantities/kinetic_energy': array([4.62843853]),
 'md/compute/ThermodynamicQuantities/volume': array([808.50548218]),
 'md/compute/ThermodynamicQuantities/pressure': array([0.00051519]),
 'particles/EllipsCustomForce/forces': array([[ 1.97188222, -0.20289414,  0.06983446],
        [-1.9971689 ,  0.25731409, -0.07120445]]),
 'particles/EllipsCustomForce/torques': array([[-1.77881997e-02, -1.32438332e-01,  8.45176692e-05],
        [-1.75892152e-02, -2.04224586e-02,  6.46268502e-02]])}

In [23]:
log_path

'/home/marjanalbooyeh/logs/pps_CG/2023-03-08-17:05:21/'

with sim._state.cpu_local_snapshot as snap:
    print(snap.particles.potential_energy)